In [1]:
from utils.data_loader import train_data_loader, test_data_loader, data_generator
from utils.inference_tools import pred_to_binary, export_csv, making_result,  error_check
from utils.model_stacking import *
from utils.cube_tools import *
from utils.model_ml import *
import vecstack
from glob import glob

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression, Lasso, RidgeClassifier, SGDClassifier, Lars, LassoLars
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import fbeta_score, make_scorer

from keras.models import Sequential, model_from_json
from keras.layers import Dense, Dropout, Conv3D, Flatten, pooling
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils 
from imgaug import augmenters as iaa

import pandas as pd
import numpy as np
import pickle
import datetime

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ubuntu/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
Using TensorFlow backend.
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# Load Data and Pre-processing

In [2]:
# Setting
path = "./data"
pos_dir = path+"/train/positive/"
neg_dir = path+"/train/negative/"

save_dir = path+"/model/"
test_dir = path+'/test/'

features = ['firstorder', 'shape']
target_voxel = (0.65, 0.65, 3)

In [ ]:
norm = 'new'
do_resample = True
do_shuffle = True
do_minmax = True

X_train, y_train = train_data_loader(pos_dir, neg_dir, norm, do_resample, do_shuffle, do_minmax, features, target_voxel, path=path)

In [ ]:
norm = 'new'
do_resample = True
do_minmax = True

X_test, patient_num, error_patient = test_data_loader(test_dir, norm, do_resample, do_minmax, features, target_voxel, path=path)

<br><br><br>

# Modeling

In [ ]:
X_train.shape

### MLP

In [ ]:
MLP = dl_mlp(X_train, y_train, num_units=256, hidden_layers=3, epochs=30, loss="cross_entropy_loss")

### CNN

In [2]:
epochs = 1
batch_size = 4
cube_shape = (32, 32, 16)
mode = 'train'
norm="new"
target_voxel = (0.65, 0.65, 3)

In [3]:
seq = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontally flip 50% of the images
    iaa.Flipud(0.2), # vertically flip 20% of the images
    iaa.GaussianBlur(sigma=(0, 3.0)), # blur images with a sigma of 0 to 3.0
    iaa.Affine(rotate=(-10, 10))
], random_order = True)

In [3]:
path = './data'

data_dir = sorted(glob(os.path.join(path, mode, '*', '*')))
data_dir, error_patient = error_check(data_dir)
data_gen = data_generator(batch_size, mode, data_dir, cube_shape, norm, target_voxel, seq=None)

In [ ]:
CNN = dl_cnn(data_gen, cube_shape=(32,32,16), batch_size=4, epochs=1, seq=None)

# Save

In [ ]:
MLP.model.save_weights(path+'/model/MLP.h5')
with open(path+'/model/MLP.json', 'w') as f :
    f.write(MLP.model.to_json())

In [ ]:
CNN.model.save_weights(path+'/model/CNN.h5')
with open(path+'/model/CNN.json', 'w') as f :
    f.write(CNN.model.to_json())

# Loading & Prediction

In [ ]:
with open(path+'/model/MLP.json', 'r') as f :
    MLP = model_from_json(f.read())
MLP.model.load_weights(path+'/model/MLP.h5')

In [ ]:
with open(path+'/model/CNN.json', 'r') as f :
    CNN = model_from_json(f.read())
CNN.model.load_weights(path+'/model/CNN.h5')

<br><br>

In [ ]:
threshold = "auto"
mode="test"
path = './data'

data_dir = sorted(glob(os.path.join(path, mode, '*')))
data_dir, error_patient = error_check(data_dir)
data_gen = data_generator(batch_size, mode, data_dir, cube_shape, norm, target_voxel, seq)

In [ ]:
y_pred_lst = []
y_pred_binary_lst =[]

pred = MLP.predict_proba(X_test)[:, 1]
y_pred_lst.append(pred)
y_pred_binary_lst.append(pred_to_binary(pred, threshold = threshold))
    
pred = CNN.predict_generator(data_gen, steps=255//batch_size, verbose=1)
y_pred_lst.append(pred)
y_pred_binary_lst.append(pred_to_binary(pred, threshold = threshold))  

final, final_df = export_csv(patient_num, error_patient, y_pred_binary_lst, y_pred_lst, path = path, index=1)
print(final_df)